In [8]:
import pandas as pd 
from pathlib import Path

In [9]:
# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "data")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

data= pd.read_csv(Path(DATA_DIR, "indices_QA_commune_IDF_2017.csv"), sep=",")

Work directory: c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet 
Data directory: c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet\data


In [10]:
#dimensions de la base et nombre de valeurs uniques de ninsee
print(f" {data.shape} \n {data['ninsee'].nunique()} ") 

 (476485, 5) 
 1309 


In [11]:
#les valeurs des indices de l'aire sont deja specifié dans la data set 
#donc il suffit de faire un filtre 
# Départements de la région parisienne (Île-de-France)
idf_departments = [75, 77, 78, 91, 92, 93, 94, 95]

# Filtrer les données par les codes INSEE des départements de l'Île-de-France
idf_data = data[data['ninsee'].isin(idf_departments)]

# Afficher les premières lignes des données filtrées
print(idf_data.head())


         date  ninsee  no2  o3  pm10
1  01/01/2017      75   23   8    38
2  01/01/2017      77   19  17    44
3  01/01/2017      78   16  15    32
4  01/01/2017      91   14  16    34
5  01/01/2017      92   23  12    32


In [12]:
# Supprimer la colonne 'date'
idf_data = idf_data.drop(columns=['date'])

Nous allons résumer ces données journalières en données annuelles par département. 
Au regard des analyses effectuées dans la 1ere partie de l'analyse descriptive, les distributions respectives de 'pm10', 'o3' et 'no2' nous indiquent des asymétrie pour 'pm10' et 'no2'. Nous résumerons donc l'information par la moyenne ou par la médiane.

In [13]:

# Résumer les données annuelles par département pour chaque variable
department_means = idf_data.groupby(['ninsee'])[['pm10', 'o3', 'no2']].agg({
    'pm10': 'median',  # Médiane pour pm10 (distribution étalée à droite)
    'o3': 'mean',      # Moyenne pour o3 (distribution presque normale)
    'no2': 'median'    # Médiane pour no2 (distribution étalée à droite)
}).reset_index()

# Afficher les premières lignes du résumé
print(department_means)


   ninsee  pm10         o3   no2
0      75  29.0  29.652055  36.0
1      77  29.0  35.389041  30.0
2      78  26.0  35.106849  27.0
3      91  27.0  34.536986  29.0
4      92  29.0  31.043836  33.0
5      93  29.0  32.013699  36.0
6      94  29.0  31.408219  34.0
7      95  27.0  35.342466  31.0


In [14]:

# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "data")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

# Vérifier si le dossier "data" existe
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Le dossier 'data' n'existe pas dans : {DATA_DIR}")

# Définir le chemin du fichier à enregistrer
file_path = DATA_DIR / "department_means.csv"

# Sauvegarder le DataFrame dans le fichier CSV
department_means.to_csv(file_path, index=False)

print(f"Fichier sauvegardé avec succès dans {file_path}.")


Work directory: c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet 
Data directory: c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet\data
Fichier sauvegardé avec succès dans c:\Users\HP\Documents\Cours 2AD ENSAE\Semestre 1\Python pour data science\indice de l'air\projet-Python\projet\data\department_means.csv.
